<a href="https://colab.research.google.com/github/Pramsi/Can-t_tame_that_Python_Semester_Project/blob/main/Can't_tame_that_Python_Semester_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [70]:
#for data manipulation
import numpy as np
import pandas as pd

#for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

#Other tooling
import os
from dotenv import load_dotenv

Import Datafiles

In [40]:
# create kaggle directory and add kaggle.json
from google.colab import files
files.upload()

! mkdir -p ~/.kaggle #create kaggle directory
! mv kaggle.json ~/.kaggle/ #move json file there
! chmod 600 ~/.kaggle/kaggle.json #change permission of the file

Saving kaggle.json to kaggle.json


In [38]:
#upload brexit dataset
!kaggle datasets download -d visalakshiiyer/twitter-data-brexit
! mkdir brexitTweets #create folder with files
! unzip twitter-data-brexit.zip -d brexitTweets #unzip data and save in folder

 99% 70.0M/70.9M [00:00<00:00, 130MB/s]
100% 70.9M/70.9M [00:00<00:00, 118MB/s]
Archive:  twitter-data-brexit.zip
  inflating: brexitTweets/TweetDataset_AntiBrexit_Jan-Mar2022.csv  
  inflating: brexitTweets/TweetDataset_ProBrexit_Jan-Mar2022.csv  


In [74]:
#create .env file
%%writefile .env
KAGGLE_USERNAME=yvonnegassner
KAGGLE_KEY=1d3b99e46686a403cd5e436aedbf7e9b

Writing .env


In [77]:
# Load environment variables from .env
load_dotenv()

True

In [78]:
#create gitignore
%%writefile .gitignore
.kaggle/
.env/

Writing .gitignore


.  ..  brexitTweets  .config  .env  .gitignore	twitter-data-brexit.zip
